In [43]:
import nbimporter
import pandas as pd
import numpy as np
import strategies_master
import inspect
import warnings
warnings.filterwarnings('ignore')
import importlib
importlib.reload(strategies_master)

<module 'strategies_master' from 'strategies_master.ipynb'>

In [3]:
raw_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Technical_Analysis_automated\data\raw\stock_data_raw.csv.gz', compression='gzip')
raw_data['Date'] = pd.to_datetime(raw_data['Date'], format='mixed')
raw_data = raw_data.drop_duplicates()
list_of_stocks = raw_data['Stock Name'].unique().tolist()

In [71]:
def all_strategy_output(list_of_stocks):
    final_output = pd.DataFrame()
    for stock in list_of_stocks:
        data = raw_data.loc[raw_data['Stock Name']==stock,]
        collector = pd.DataFrame()
        for name, func in inspect.getmembers(strategies_master, inspect.isfunction):
            if name.startswith("strategy"):
                result = func(data) 
                result['strategy_name'] = name
                collector = pd.concat([collector, result])
        final_output = pd.concat([final_output, collector])
        final_output['Position'].replace(-1, 0, inplace=True)
        final_output = final_output.fillna(0)

    final_output = final_output.pivot_table(index=['Date', 'Stock Name'], columns='strategy_name', values='Position').reset_index()
    return(final_output)

In [86]:
def all_strategy_output_optimized(list_of_stocks):
    stock_data_dict = {stock: raw_data.loc[raw_data['Stock Name'] == stock] for stock in list_of_stocks}
    final_output = []
    
    for stock in list_of_stocks:
        data = stock_data_dict[stock]
        collector = []

        # Iterate over all strategy functions
        for name, func in inspect.getmembers(strategies_master, inspect.isfunction):
            if name.startswith("strategy"):
                result = func(data)
                result['strategy_name'] = name
                collector.append(result)

        # Concatenate all strategy results for the current stock
        if collector:
            collector_df = pd.concat(collector, ignore_index=True)
            final_output.append(collector_df)

    # Concatenate all stock results
    if final_output:
        final_output = pd.concat(final_output, ignore_index=True)
        
        # Replace -1 with 0 and fill NaNs
        final_output['Position'].replace(-1, 0, inplace=True)
        final_output = final_output.fillna(0)

        # Pivot the data to the desired format
        final_output = final_output.pivot_table(index=['Date', 'Stock Name'], columns='strategy_name', values='Position').reset_index()

    return final_output


In [90]:
backtester_input = all_strategy_output_optimized(list_of_stocks)

reliance.NS
tcs.NS
hdfcbank.NS
icicibank.NS
bhartiartl.NS
sbin.NS
infy.NS
lici.NS
itc.NS
hindunilvr.NS
lt.NS
bajfinance.NS
hcltech.NS
maruti.NS
sunpharma.NS
adanient.NS
kotakbank.NS
titan.NS
ongc.NS
tatamotors.NS
ntpc.NS
axisbank.NS
dmart.NS
adanigreen.NS
adaniports.NS
ultracemco.NS
asianpaint.NS
coalindia.NS
bajajfinsv.NS
bajaj-auto.NS
powergrid.NS
nestleind.NS
wipro.NS
m&m.NS
ioc.NS
jiofin.NS
hal.NS
dlf.NS
adanipower.NS
jswsteel.NS
tatasteel.NS
siemens.NS
irfc.NS
vbl.NS
zomato.NS
pidilitind.NS
grasim.NS
sbilife.NS
bel.NS
ltim.NS
trent.NS
pnb.NS
indigo.NS
bankbaroda.NS
hdfclife.NS
abb.NS
bpcl.NS
pfc.NS
godrejcp.NS
tatapower.NS
hindalco.NS
hindzinc.NS
techm.NS
ambujacem.NS
indusindbk.NS
cipla.NS
gail.NS
recltd.NS
britannia.NS
unionbank.NS
adaniensol.NS
iob.NS
lodha.NS
eichermot.NS
canbk.NS
tataconsum.NS
drreddy.NS
tvsmotor.NS
zyduslife.NS
atgl.NS
vedl.NS
cholafin.NS
havells.NS
heromotoco.NS
dabur.NS
shreecem.NS
mankind.NS
bajajhldng.NS
divislab.NS
apollohosp.NS
nhpc.NS
shriramfin.NS
bo

ashapurmin.NS
hgs.NS
muthootmf.NS
subros.NS
ramky.NS
sunflag.NS
carerating.NS
gensol.NS
skipper.NS
laopala.NS
lumaxtech.NS
dcxindia.NS
bomdyeing.NS
hikal.NS
jisljaleqs.NS
cupid.NS
avalon.NS
luxind.NS
nucleus.NS
tastybite.NS
sotl.NS
arvsmart.NS
sandhar.NS
salasar.NS
neogen.NS
datamatics.NS
jtlind.NS
anup.NS
heritgfood.NS
thyrocare.NS
vadilalind.NS
navnetedul.NS
dishtv.NS
kddl.NS
kalamandir.NS
landmark.NS
indoco.NS
bajajcon.NS
tvssrichak.NS
cartrade.NS
sbcl.NS
fiemind.NS
prakash.NS
deltacorp.NS
rajratan.NS
ideaforge.NS
mahlog.NS
pfocus.NS
greavescot.NS
dollar.NS
uflex.NS
unitech.NS
bfutilitie.NS
shardacrop.NS
banarisug.NS
sequent.NS
greenply.NS
maithanall.NS
shk.NS
sunclay.NS
guficbio.NS
blse.NS
diacabs.NS
esafsfb.NS
vsttillers.NS
hleglas.NS
bcg.NS
goodluck.NS
swarajeng.NS
seamecltd.NS
smlisuzu.NS
ashiana.NS
dalmiasug.NS
hindwareap.NS
sagcem.NS
saksoft.NS
apollo.NS
supriya.NS
autoaxles.NS
stylamind.NS
flair.NS
vindhyatel.NS
carysil.NS
thejo.NS
mpsltd.NS
marathon.NS
filatex.NS
nrbbearing.

felix.NS
nippobatry.NS
premierpol.NS
vardhacrlc.NS
rppinfra.NS
higreen.NS
surani.NS
nilainfra.NS
kaya.NS
sjlogistic.NS
crayons.NS
nitco.NS
eurobond.NS
medico.NS
alankit.NS
emamireal.NS
saakshi.NS
sejalltd.NS
shemaroo.NS
bewltd.NS
ifbagro.NS
dicind.NS
precot.NS
asahisong.NS
orbtexp.NS
modisonltd.NS
aurum.NS
kdl.NS
exxaro.NS
karnika.NS
sahyadri.NS
skp.NS
kcpsugind.NS
mkpl.NS
manaksteel.NS
univphoto.NS
scpl.NS
sigma.NS
izmo.NS
sintercom.NS
poddarment.NS
plazacable.NS
dcmnvl.NS
shyamcent.NS
sakhtisug.NS
esfl.NS
atmastco.NS
kore.NS
lykalabs.NS
starpaper.NS
srghfl.NS
hindmotors.NS
ponnierode.NS
ashimasyn.NS
kotharipro.NS
gokul.NS
hitechcorp.NS
digikore.NS
knagri.NS
uravi.NS
lawsikho.NS
goldstar.NS
tcl.NS
ruchira.NS
mahepc.NS
prov.NS
osiahyper.NS
mawanasug.NS
rhl.NS
nathbiogen.NS
eiffl.NS
sartele.NS
vipulltd.NS
essentia.NS
mwl.NS
ucal.NS
pasuptac.NS
drone.NS
infinium.NS
coastcorp.NS
mos.NS
bil.NS
indterrain.NS
aries.NS
shreerama.NS
dynpro.NS
shera.NS
mhlxmiru.NS
ndlventure.NS
gulfpetro.NS
tak

amiable.NS
umesltd.NS
omkarchem.NS
arenterp.NS
bkmindst.NS
nirajispat.NS
innovative.NS
accord.NS
shyamtel.NS
smvd.NS
dcmfinserv.NS
premier.NS
creativeye.NS
ahimsa.NS
alpsindus.NS
melstar.NS
jalan.NS
sabevents.NS
sanco.NS
lakpre.NS
vasa.NS
cmmipl.NS


In [94]:
backtester_input.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Technical_Analysis_automated\backtester_input.csv', index=False, compression='gzip')